In [1]:
import os
os.chdir("../../crystal/")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

from tqdm import tqdm
from math import lcm
from typing import List, Dict
from ripser import Rips
from utils import wasserstein_distance_matrix, plot_distance_matrix

from Symmetry import Symmetry
from UnitCell import UnitCell
from RandomCrystal import RandomCrystal
from Fractional import FractionalCoordinate, FractionalCoordinateList
from Positional import PositionalCoordinate, PositionalCoordinateList

from sklearn.manifold import TSNE, Isomap, MDS, SpectralEmbedding
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

plt.rcParams['text.usetex'] = False

In [2]:
df = pd.read_csv("output/space_groups.csv", sep=" ")
df

,Space Group,Crystal System,Asymmetric Unit,Symmetries,Group Order,Unit Cell
0,2,Triclinic,30,"x,y,z;-x,-y,-z",2,"13.027,11.1822,9.2189,92.35,107.3,104.5"
1,9,Monoclinic,71,"x,y,z;x,-y,z+1/2;x+1/2,y+1/2,z;x+1/2,-y+1/2,z+1/2",4,"20.408,13.304,20.166,90.0,102.33,90.0"
2,10,Monoclinic,50,"x,y,z;-x,y,-z;-x,-y,-z;x,-y,z",4,"12.7411,12.6989,20.9991,90.0,96.29,90.0"
3,11,Monoclinic,25,"x,y,z;-x,y+1/2,-z;-x,-y,-z;x,-y+1/2,z",4,"11.454,21.695,7.227,90.0,93.15,90.0"
4,12,Monoclinic,24,"x,y,z;-x,y,-z;-x,-y,-z;x,-y,z;x+1/2,y+1/2,z;-x...",8,"22.684,13.373,12.553,90.0,69.48,90.0"
...,...,...,...,...,...,...
68,223,Cubic,7,"x,y,z;-x,-y,z;x,-y,-z;-x,y,-z;z,x,y;y,z,x;-z,-...",48,"13.705,13.705,13.705,90.0,90.0,90.0"
69,225,Cubic,4,"x,y,z;-x,-y,z;x,-y,-z;-x,y,-z;z,x,y;y,z,x;-z,-...",192,"13.624,13.624,13.624,90.0,90.0,90.0"
70,227,Cubic,5,"x,y,z;-x+1/4,-y+1/4,z;x,-y+1/4,-z+1/4;-x+1/4,y...",192,"24.345,24.345,24.345,90.0,90.0,90.0"
71,229,Cubic,5,"x,y,z;-x,-y,z;x,-y,-z;-x,y,-z;z,x,y;y,z,x;-z,-...",96,"18.578,18.578,18.578,90.0,90.0,90.0"


# Orthorhombic vs Tetragonal

In [15]:
crystal_system_1: str = 'Orthorhombic'
crystal_system_2: str = 'Tetragonal'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 96

Least common multiple: 16


In [16]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:05<00:00,  9.83it/s]


In [17]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:04<00:00, 11.19it/s]


In [18]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:01<00:00, 55.22it/s]


In [19]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,1.183376,-2.341149,0.557051,0.864894,0.488278,Orthorhombic,42_1
1,0.114120,-3.048407,0.569660,-0.119038,0.432919,Orthorhombic,42_2
2,-0.925338,0.201007,-1.363585,0.113157,0.547060,Orthorhombic,42_3
3,-1.320330,-0.273614,0.308607,-1.009594,1.297252,Orthorhombic,42_4
4,0.428655,0.363391,0.537682,-1.228568,0.924752,Orthorhombic,42_5


In [20]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 0.87
Classification Report:
              precision    recall  f1-score   support

Orthorhombic       0.84      0.94      0.89        17
  Tetragonal       0.91      0.77      0.83        13

    accuracy                           0.87        30
   macro avg       0.88      0.86      0.86        30
weighted avg       0.87      0.87      0.86        30



# Orthorhombic vs Trigonal

In [21]:
crystal_system_1: str = 'Orthorhombic'
crystal_system_2: str = 'Trigonal'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 96

Least common multiple: 24


In [22]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:04<00:00, 11.15it/s]


In [23]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:04<00:00, 11.18it/s]


In [24]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:01<00:00, 62.74it/s]


In [25]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,1.059657,0.095260,-0.396880,-0.628736,0.194854,Orthorhombic,58_1
1,0.244913,-0.960588,-0.240363,0.304085,-0.606019,Orthorhombic,58_2
2,-0.456868,0.064246,0.007675,-0.514546,0.414383,Orthorhombic,58_3
3,-1.032271,-0.161264,-0.121738,0.708544,0.339098,Orthorhombic,58_4
4,0.183588,-0.561426,0.254413,0.345948,0.228789,Orthorhombic,58_5


In [26]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 1.00
Classification Report:
              precision    recall  f1-score   support

Orthorhombic       1.00      1.00      1.00        17
    Trigonal       1.00      1.00      1.00        13

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



# Orthorhombic vs Hexagonal

In [27]:
crystal_system_1: str = 'Orthorhombic'
crystal_system_2: str = 'Hexagonal'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 96

Least common multiple: 24


In [28]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:04<00:00, 10.74it/s]


In [29]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:04<00:00, 10.68it/s]


In [30]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:02<00:00, 48.95it/s]


In [31]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,-0.062228,-0.179955,-0.500768,0.837415,-0.320592,Orthorhombic,61_1
1,-0.059903,-0.937704,1.395901,1.473715,-0.883569,Orthorhombic,61_2
2,0.651274,-0.346762,1.243240,0.112221,1.021925,Orthorhombic,61_3
3,-0.122919,-0.129478,-0.648418,0.724762,0.167140,Orthorhombic,61_4
4,0.461349,-0.198212,1.165180,1.444028,0.679423,Orthorhombic,61_5


In [32]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 0.97
Classification Report:
              precision    recall  f1-score   support

Orthorhombic       0.93      1.00      0.96        13
   Hexagonal       1.00      0.94      0.97        17

    accuracy                           0.97        30
   macro avg       0.96      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



# Orthorhombic vs Cubic

In [39]:
crystal_system_1: str = 'Orthorhombic'
crystal_system_2: str = 'Cubic'

space_groups_1: np.ndarray = df.loc[df['Crystal System'] == crystal_system_1]['Space Group'].unique()
space_groups_2: np.ndarray = df.loc[df['Crystal System'] == crystal_system_2]['Space Group'].unique()

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_1), size = 1, dtype = int)[0]
space_group_1: int = space_groups_1[random_space_group_index]

random_space_group_index: np.ndarray = np.random.randint(low = 0, high = len(space_groups_2), size = 1, dtype = int)[0]
space_group_2: int = space_groups_2[random_space_group_index]

row_1 = df.loc[df['Space Group'] == space_group_1]
row_2 = df.loc[df['Space Group'] == space_group_2]

group_order_1: int = row_1['Group Order'].values[0]
group_order_2: int = row_2['Group Order'].values[0]

least_common_multiple: int = lcm(group_order_1, group_order_2)
print(f'Least common multiple: {least_common_multiple}')

crystal_dict = {}
n = 96

Least common multiple: 24


In [40]:
symmetries: list = row_1['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_1['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_1
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_1}_{_+1}'] = {
        'system': crystal_system_1,
        'persistence': persistence
    }

100%|██████████| 50/50 [00:04<00:00, 11.60it/s]


In [41]:
symmetries: list = row_2['Symmetries'].values[0].split(sep=";")
symmetries: List[Symmetry] = [Symmetry(sym) for sym in symmetries]

unit_cell: List[str] = row_2['Unit Cell'].values[0].split(sep=",")
unit_cell: UnitCell = UnitCell(*[float(x) for x in unit_cell])
normalised_unit_cell: UnitCell = unit_cell.normalise()

normalising_constant: float = unit_cell.normalising_constant
k = least_common_multiple // group_order_2
if least_common_multiple < n:
    k *= (n // least_common_multiple)

rips = Rips(maxdim=2, verbose=False)

for _ in tqdm(range(50)):
    random_crystal: RandomCrystal = RandomCrystal(symmetries, k)
    positional_coordinates: PositionalCoordinateList = random_crystal.fractional_coordinates.orthogonalise(unit_cell)
    normalised_positional_coordinates: PositionalCoordinateList = positional_coordinates.normalise(normalising_constant)

    distance_matrix: np.ndarray = normalised_positional_coordinates.calculate_distance_matrix(normalised_unit_cell, boundary_conditions=True)
    persistence = rips.fit_transform(X = distance_matrix, distance_matrix=True)

    for dim, intervals in enumerate(persistence):
        persistence[dim] = np.array(list(filter(lambda i: i[1] < float('inf'), intervals)))

    crystal_dict[f'{space_group_2}_{_+1}'] = {
        'system': crystal_system_2,
        'persistence': persistence
    }

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:06<00:00,  7.35it/s]


In [42]:
distance_matrix_0 = wasserstein_distance_matrix(crystal_dict, 0)
distance_matrix_1 = wasserstein_distance_matrix(crystal_dict, 1)
distance_matrix_2 = wasserstein_distance_matrix(crystal_dict, 2)

distance_matrix: np.ndarray = np.maximum.reduce([distance_matrix_0, distance_matrix_1, distance_matrix_2])

100%|██████████| 100/100 [00:02<00:00, 41.97it/s]


In [43]:
system = [v['system'] for v in crystal_dict.values()]
crystals = list(crystal_dict.keys())

n = 5
mds = MDS(n_components=n, dissimilarity='precomputed', metric=True)
embedding = mds.fit_transform(distance_matrix)

embedding_df = pd.DataFrame(embedding, columns=[str(_+1) for _ in range(n)])
embedding_df['Crystal System'] = system
embedding_df['Name'] = crystals

X = embedding
y = embedding_df['Crystal System'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

embedding_df.head()

,1,2,3,4,5,Crystal System,Name
0,-0.060484,-1.735034,0.877962,-0.451612,0.612893,Orthorhombic,44_1
1,0.794545,-0.835849,0.961011,-0.742474,-0.127714,Orthorhombic,44_2
2,0.645459,-0.052235,-0.589091,-0.564019,0.219467,Orthorhombic,44_3
3,0.074939,-0.752232,0.137763,0.500420,-0.061140,Orthorhombic,44_4
4,-0.261179,-1.611209,1.164495,-0.057774,1.007767,Orthorhombic,44_5


In [44]:
clf = OneVsOneClassifier(SVC())
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

class_report = classification_report(y_test, y_pred, target_names = embedding_df['Crystal System'].unique())
print('Classification Report:')
print(class_report)

Accuracy: 0.93
Classification Report:
              precision    recall  f1-score   support

Orthorhombic       0.92      0.92      0.92        13
       Cubic       0.94      0.94      0.94        17

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30

